## 영화별 줄거리 내  키워드 추출<br>
줄거리 기준 : 네이버 영화 페이지의 줄거리

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen

import re
from konlpy.tag import Okt, Kkma, Komoran
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

import networkx
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec
from difflib import ndiff

import os
os.environ['JAVA_OPTS'] = 'Xmx8192m'

Using TensorFlow backend.
c:\users\leejb\appdata\local\programs\python\python36\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [9]:
full_execute = False

In [2]:
df = pd.read_csv('./naver_movie_dataset/naver_movie_dataset_1227.csv')

In [3]:
df.drop(df[df['plot'].isnull()].index, inplace=True)

df['plot'] = df['plot'].apply(str)
plots = df['plot'].copy()#pd.Series(df['plot'])

df['plot'] = list(map(lambda x:re.sub("\‘[가-힣 ]+\’(\([가-힣 ]+\))?[^\s]*"," ",x),df['plot']))
df['plot'] = list(map(lambda x:re.sub("[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)[^\s]*"," ",x), df['plot']))

for idx in tqdm(df.index):
    chars1, chars2 = [],[]
    
    chars1 = list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])]))
    if chars1:
        if len(chars1) == 1:
            regex1 = ''.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        else:
            regex1 = '|'.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        #print(regex1)
        plots[idx] = re.sub('(%s)[^\s]'%(regex1),' ',df.loc[idx,'plot'])

    chars2 = ''.join(re.findall('[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)',plots[idx]))
    if chars2:
        regex2 = '|'.join(map(str.strip, re.compile('\([가-힣 ]+\)').split(chars2)[:-1]))
        regex2 += '|'+' '.join(regex2.split('|')).replace(' ','|')
        #print(regex2)
        plots[idx] = re.sub('(%s)[^\s]'%(regex2),' ',df.loc[idx,'plot'])

df['plot'] = plots

In [4]:
del plots

In [5]:
display(len(df['plot']))
df['plot'].head(10)

10255

0    에디슨의 마지막 발명품, 유령 탐지기? “증명할 수 없는 미스터리란 없다” 가지 말...
1    멈추지 않는 성욕과 사랑하는 세 명의 남편을 위해,\r 선상에서 밤낮 없이 몸을 파...
2    동물적인 직감과 1초의 오차도 용납하지 않는 설계자  \r 철저한 계획 하에 움직이...
3    어렸을 적부터 함께 자라고, IT 회사도 공동 운영,\r 사는 집도 아랫집, 윗집으...
4    안락사, 인공지능과 완전히 통제된 교육,\r 디지털 유산과 알 권리, 환경오염, 전...
5    제2차 세계대전 나치에 점령된 노르웨이.\r 나치를 함락시킬 유일한   노르웨이 저...
6    담배를 사러 다녀 오겠다는 아버지는 그렇게 돌아오지 않았다.\r 엄마는 아버지 대신...
7    100년 전, 소녀들이 대한독립을 위해 세상에 맞서기 시작했다.\r 우리가 기억해야...
8    가상현실 기술의 발달로 환상과 현실이 혼합된 근 미래의 유럽.\r 어느 날, 사이버...
9    물에 잠긴 고대 마야의 수중도시 '시발바'를 향해\r 짜릿한 동굴 다이빙에 나선 '...
Name: plot, dtype: object

In [6]:
df['plot'] = df['plot'].str.replace('\r\xa0', ' ')
display(df[df['plot'].str.contains('\r')].index)
df['plot'] = df['plot'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .][^\s]+"," ")

Int64Index([], dtype='int64')

In [7]:
df['plot'].head(20)

0     에디슨의 마지막 발명품, 유령 탐지기?  수 없는 미스터리란 없다”  말라는 곳에 ...
1     멈추지 않는 성욕과 사랑하는 세 명의 남편을 위해, 선상에서 밤낮 없이 몸을 파는 ...
2     동물적인 직감과   오차도 용납하지 않는 설계자   철저한 계획 하에 움직이는 기술...
3     어렸을 적부터 함께 자라고,   회사도 공동 운영, 사는 집도 아랫집, 윗집으로 이...
4     안락사, 인공지능과 완전히 통제된 교육, 디지털 유산과 알 권리, 환경오염, 전쟁을...
5     제  세계대전 나치에 점령된 노르웨이. 나치를 함락시킬 유일한   노르웨이 저항군 ...
6     담배를 사러 다녀 오겠다는 아버지는 그렇게 돌아오지 않았다. 엄마는 아버지 대신 일...
7       전, 소녀들이 대한독립을 위해 세상에 맞서기 시작했다. 우리가 기억해야 할 그 ...
8     가상현실 기술의 발달로 환상과 현실이 혼합된 근 미래의 유럽. 어느 날, 사이버 세...
9     물에 잠긴 고대 마야의 수중도시   향해 짜릿한 동굴 다이빙에 나선   친구들은 예...
10    어느 날 갑자기 세상이 사라졌다! 하루아침에 모두 붕괴되어 버린 건물들과 사라져버린...
11    터키 국경 지역에서 잔악무도한 테러 단체가 판치는 가운데, 알파슬란 대위가 이끄는 ...
12    군인들이 어떻게 생활하는지, 그들의 인간적인 모습을 알리고 싶어서 미군부대를 따라 ...
13      봄에 태어나  딸이자 아내, 동료이자 엄마로  오늘을 살아가는    어딘가 갇힌...
14      이어진 이란  전쟁의 영웅 아흐마드 모테바젤. 지휘관이자 혁명가인 그는 폭탄이 ...
15    번개맨, 번개걸, 뿡뿡이, 뿡순이, 뚜앙 등   캐릭터들이 최초로 한 자리에 총 출...
16    나를 세상에 태어나게 한    고소하고 싶어요..."   없이 살아온 어쩌면   소년  
17    전 여친에 상처받은     때처럼 숙취로 시작한 아침,   번호의 누군가와 

In [8]:
okt, kkma, kmr = Okt(), Kkma(), Komoran()

### 단어 출현 빈도

In [11]:
from collections import Counter

In [12]:
cnt = Counter()

for plot in tqdm(df['plot']):
    for word,tag in kmr.pos(plot):
        if tag in ['NNP', 'NNG', 'VV']:
            if not cnt[(word,tag)]:
                cnt[(word,tag)] = 1
            else:
                cnt[(word,tag)] += 1

In [13]:
cnt.most_common()

[(('되', 'VV'), 16300),
 (('하', 'VV'), 8294),
 (('있', 'VV'), 7486),
 (('위하', 'VV'), 6095),
 (('자신', 'NNG'), 5437),
 (('시작', 'NNG'), 5070),
 (('사람', 'NNG'), 3724),
 (('사랑', 'NNG'), 3594),
 (('받', 'VV'), 3394),
 (('알', 'VV'), 3350),
 (('날', 'NNG'), 3289),
 (('만나', 'VV'), 3079),
 (('살', 'VV'), 2774),
 (('친구', 'NNG'), 2642),
 (('찾', 'VV'), 2588),
 (('속', 'NNG'), 2377),
 (('보', 'VV'), 2310),
 (('집', 'NNG'), 2293),
 (('가', 'VV'), 2230),
 (('사건', 'NNG'), 2157),
 (('남자', 'NNG'), 1993),
 (('일', 'NNG'), 1972),
 (('빠지', 'VV'), 1934),
 (('대하', 'VV'), 1851),
 (('떠나', 'VV'), 1759),
 (('후', 'NNG'), 1683),
 (('가족', 'NNG'), 1668),
 (('오', 'VV'), 1612),
 (('여자', 'NNG'), 1599),
 (('이야기', 'NNG'), 1581),
 (('마음', 'NNG'), 1560),
 (('말', 'NNG'), 1529),
 (('아버지', 'NNG'), 1506),
 (('마을', 'NNG'), 1462),
 (('세상', 'NNG'), 1449),
 (('삶', 'NNG'), 1448),
 (('사실', 'NNG'), 1393),
 (('세계', 'NNG'), 1367),
 (('보내', 'VV'), 1346),
 (('시간', 'NNG'), 1319),
 (('느끼', 'VV'), 1308),
 (('앞', 'NNG'), 1297),
 (('발견', 'NNG'), 1261),


In [14]:
cnt_df = pd.DataFrame(np.hstack(np.array(cnt.most_common())[:,0]).reshape(-1,2),
                     columns=['word','tag'])
cnt_df['cnt'] = np.array(cnt.most_common())[:,1]
cnt_df.head()

,word,tag,cnt
0,되,VV,16300
1,하,VV,8294
2,있,VV,7486
3,위하,VV,6095
4,자신,NNG,5437


### 불용어 제거

In [15]:
from nltk.corpus import stopwords
if full_execute:
    stopwords.words('english')

In [16]:
idx_cnt=50

In [17]:
if full_execute:
    idx_cnt += 50
    cnt_df[cnt_df['tag'].str.startswith('N')][idx_cnt:idx_cnt+50]

#### 수동

In [22]:
'''
cnt_idx = 1300, cnt < 66
'''
ridx = [4, 6, 10, 15, 25, 41, 45, 60, 61, 64, 78, 90, 91, 96, 97, 102, 107, 109, 140, 145, 160, 161, 162, 186, 188,
205, 221, 234, 249, 253, 260, 268, 271, 272, 280, 281, 282, 284, 292, 302, 304, 314, 316, 320, 324, 328, 356,
362, 379, 381, 399, 415, 416, 430, 434, 453, 462, 465, 466, 469, 479, 487, 495, 496, 498, 503, 508, 514, 516,
521, 525, 528, 551, 552, 555, 563, 569, 573, 576, 577, 583, 585, 599, 600, 603, 608, 610, 612, 618, 629, 631,
650, 655, 671, 674, 683, 690, 693, 700, 701, 714, 724, 728, 729, 732, 733, 739, 742, 745, 748, 750, 754, 756,
759, 764, 771, 778, 785, 788, 789, 795, 803, 826, 830, 843, 859, 866, 876, 881, 883, 901, 913, 931, 934, 939,
947, 956, 961, 965, 967, 972, 973, 978, 990, 1010, 1027, 1034, 1045, 1046, 1053, 1058, 1063, 1078, 1088, 1089, 1109, 1110, 1112, 1116, 1120, 1137, 1141, 1145, 1147, 1155, 1157, 1158, 1160, 1168, 1179, 1187, 1207,
1210, 1211, 1214, 1223, 1225, 1231, 1234, 1238, 1241, 1256, 1264, 1267, 1269, 1278, 1283, 1286, 1290, 1293,
1301, 1308, 1316, 1319, 1322, 1341, 1352, 1354, 1355, 1370, 1375, 1390, 1391, 1402, 1415, 1417, 1418, 1421,
1423, 1430, 1432, 1437, 1445, 1456, 1457, 1458, 1459, 1481, 1492, 1497, 1500, 1501, 1504, 1505, 1509, 1517,
1518, 1523, 1533, 1535, 1538, 1541, 1560, 1565, 1575, 1576, 1591, 1595, 1596, 1601, 1607, 1608, 1611, 1617,
1619, 1624, 1634, 1637, 1638, 1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721]


#### 수동 이후 부분 중 명사, 길이 = 1

In [19]:
'''
idx > 1720, tag == N, word len = 1, cnt < 66,
'''
stops_idx = cnt_df[(cnt_df['tag'].str.startswith('N')) & (cnt_df['cnt'] < 66) & (cnt_df['word'].agg(len) == 1)].index

In [23]:
print(cnt_df.loc[stops_idx][['word','tag']].index)
print(cnt_df.loc[ridx,][['word','tag']].index)

Int64Index([ 1720,  1723,  1724,  1757,  1780,  1783,  1809,  1850,  1859,
             1875,
            ...
            29168, 29207, 29693, 29748, 30559, 30952, 31000, 31076, 31142,
            31787],
           dtype='int64', length=579)
Int64Index([   4,    6,   10,   15,   25,   41,   45,   60,   61,   64,
            ...
            1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721],
           dtype='int64', length=259)


In [24]:
stops = np.concatenate((cnt_df.loc[stops_idx][['word','tag']].values, cnt_df.loc[ridx,][['word','tag']].values))

### 불용어

In [27]:
with open('./stopwords/1.txt',encoding='utf-8') as f:
    stops1 = f.readlines()
with open('./stopwords/2.txt',encoding='utf-8') as f:
    stops2 = f.readlines()
with open('./stopwords/3.txt',encoding='utf-8') as f:
    stops3 = f.readlines()

In [28]:
stops1 = list(map(lambda x: x.replace('\n',''),stops1))
stops2 = list(map(lambda x: x.replace('\n',''),stops2))
stops3 = list(map(lambda x: x.replace('\n',''),stops3))

In [29]:
stopwords = list(set(stops1) & set(stops2) & set(stops3))

### Word2Vec

#### 추출될 태그

#### Kkma

In [30]:
if full_execute:
    result_kkma = []
    for plot in tqdm(plots):
        words = []
        for word,tag in kkma.pos(plot):
            if tag in ['NNG', 'NNP', 'VV']:
                words.append(word)
        words = (' '.join(words)).strip()
        result_kkma.append(words)

In [31]:
if full_execute:
    with open('naver_plot_kkma.nlp','w',encoding='utf-8') as f:
        f.write('\n'.join(result_kkma))

    w2v_kkma = word2vec.LineSentence('./model/naver_plot_kkma.nlp')
    model_kkma = word2vec.Word2Vec(w2v_kkma, size=200, window=5, hs=1, min_count=2, sg=1)
    model_kkma.save('./model/naver_plot_kkma.model')

#### Komoran

In [32]:
result_kmr = []
for plot in tqdm(df['plot']):
    words = []
    for word,tag in komoran.pos(plot):
        #if tag in ['NNG', 'NNP', 'VV']:
        if tag in ['NNG', 'NNP']:
            words.append(word)
    words = (' '.join(words)).strip()
    result_kmr.append(words)

In [33]:
with open('./model/naver_plot_kmr.nlp','w', encoding='utf-8') as f:
    f.write('\n'.join(result_kmr))

w2v_kmr = word2vec.LineSentence('./model/naver_plot_kmr.nlp')
model_kmr = word2vec.Word2Vec(w2v_kmr, size=200, window=5, hs=1, min_count=10, sg=1)
model_kmr.save('./model/naver_plot_kmr_min10_N_only.model')

#### load & test model

In [34]:
if full_execute:
    model_kkma = word2vec.Word2Vec.load('./model/naver_plot_kkma.model')
model_kmr = word2vec.Word2Vec.load('./model/naver_plot_kmr_min10_N_only.model')

In [36]:
if full_execute:
    word = '레이싱'
    print('most similar words to',word,'in word2vec')
    print('Kkma\t\t\t\tKomoran')
    for (w1,t1),(w2,t2) in zip(model_kkma.wv.most_similar(word,topn=20), model_kmr.wv.most_similar(word,topn=20)):
        print("%-10s\t%.4f\t\t%-10s\t%.4f"%(w1,t1,w2,t2))

### Textrank

In [37]:
class RawTaggerReader:
    def __init__(self, plot, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.plot = plot
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        ch = self.rgxSplitter.split(self.plot)
        if len(ch) == 1: ch.append('.')
        for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
            if not s: continue
            yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0

    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])

    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()

        if int(len(ranks) * ratio) > 10:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        else:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:10]
        pairness = {}
        startOf = {}
        tuples = {}

        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if len(k) == 1:
                if used.intersection(set(k)): continue
                both[k] = tuples[k]
                for w in k:
                    used.add(w)
        
        return both

### 키워드 추출

In [128]:
results = []
for idx in tqdm(df.index):
    keywords = []
    tr = TextRank(window=5, coef=1)
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV'),
                   ('위하', 'VV')]).union(set(map(lambda x: tuple(x),stops)))
    tr.load(RawTaggerReader(df.loc[idx,'plot']),
            lambda w:w not in stopword and w[0] not in stopwords and
            w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP')))
            #and w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP', 'VV')))
    tr.build()
    kw = tr.extract(0.2)
    for k in sorted(kw, key=kw.get, reverse=True):
        keywords.append(k[0][0])
        #print("%s\t%g" % (k, kw[k]))
    if idx %100 == 0:
        print(idx,'/10255',keywords)
    results.append(keywords)

0 /10255 ['마지막', '발명품', '유령']
100 /10255 ['말', '기사', '라디오', '장난', '카페', '기적', '시작', '가에', '화가', '사연']
200 /10255 ['신비', '요정', '세계', '위험', '마법', '기억', '마왕', '모험', '시작', '사실']
300 /10255 ['찰스', '탐정', '손녀', '소피아', '사립', '도착', '구성원', '살인', '동기', '가족']
400 /10255 ['전문가', '괴물', '첨단', '특수', '훈련', '기지', '전투', '파일럿', '로스', '박사']
500 /10255 ['마음', '작품', '목소리', '세대', '기록', '할리우드', '이야기', '블록버스터']
600 /10255 ['진입', '자리', '남자', '위협', '비밀', '출장', '최악', '일상', '컴백', '비행']
700 /10255 ['폭력', '소녀', '이사', '첫사랑', '추억', '위험', '선택', '경찰관', '아버지', '첫눈']
800 /10255 ['답장', '과거', '가게', '뮤지', '도둑', '아츠', '인연', '연결', '이름', '편지']
900 /10255 ['프리', '작업', '충격', '사이', '의구심', '인터뷰', '천국', '필리핀', '시나리오', '비밀']
1000 /10255 ['아빠', '결혼', '선배', '아가씨', '생일날', '안하무인', '사랑', '사실', '첫사랑', '짝사랑']
1100 /10255 ['임무', '가장', '행복', '무기', '거래', '가석방', '출소', '성공', '증권', '사고']
1200 /10255 ['미래', '인공지능', '로봇', '강아지', '특징', '간직', '생사', '모험', '시작', '발견']
1300 /10255 ['이모', '마음', '딸', '입사', '시험', '유미', '준비', '어머니', '전화', '집']
1400 /10255 ['

In [138]:
keyword_df = pd.DataFrame([(idx,result) for idx,result in zip(df.index,results)],columns=['idx','keywords'])

In [139]:
keyword_df.to_csv('./movie_keywords_1231.csv',index=False, encoding='utf-8')

In [140]:
del keyword_df

In [141]:
keyword_df = pd.read_csv('./movie_keywords_1231.csv')

In [148]:
keyword_df.head(10)

,idx,keywords
0,0,"['마지막', '발명품', '유령']"
1,1,"['선상', '몸', '성욕', '사랑', '남자', '운명', '남편', '여인']"
2,2,"['은행', '기술자', '직감', '용납', '멤버', '비상', '설계자', '..."
3,3,"['성사', '집으로', '집', '운영', '공동', '스마트폰', '아마존', ..."
4,4,"['디지털', '권리', '통제', '교육', '유산', '수도', '세상', '신..."
5,5,"['총상', '탈출', '유일한', '노르웨이', '저항', '작전', '발각', ..."
6,6,"['엄마', '대신', '도시락', '원망', '그리움', '형', '준비', '아..."
7,7,"['세상', '독립', '소녀', '시작', '이름', '기억', '기록']"
8,8,"['현실', '가상', '세계', '살인', '테러', '조직', '미래', '유럽..."
9,9,"['사고', '목숨', '극한', '고대', '마야', '예기', '피해', '숨바..."


In [166]:
merged = pd.merge(df,keyword_df,left_on=df.index,right_on=keyword_df.idx,)

In [174]:
merged.drop(columns=['key_0', 'idx'], inplace=True)

In [176]:
merged.to_csv('./merged.csv',index=False)